In [2]:
import sys, warnings, nltk, pandas as pd, numpy as np, feedparser, urllib.request
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from collections import Counter 
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from bs4 import BeautifulSoup
from inscriptis import get_text

In [3]:
def getRSS(rssURL):
    rss = feedparser.parse(rssURL)
    titleList = []
    linkList = []
    summaryList = []
    dateList = []
    for i in rss.entries:
        titleList.append(i.title)
        linkList.append(i.link)
        summaryList.append(i.summary)
        dateList.append(i.published)

    titles = pd.Series(titleList)
    links = pd.Series(linkList)
    dates = pd.Series(dateList)
    summaries = pd.Series(summaryList)
    df = pd.DataFrame({'title': titles, 'date': dates, 'link': links, 'summary': summaries})

    return df

In [4]:
getRSS('http://www.espn.com/espn/rss/nba/news')

,title,date,link,summary
0,"Sources: Kawhi to Toronto, DeRozan to Spurs","Wed, 18 Jul 2018 08:27:18 EST",http://www.espn.com/nba/story/_/id/24128461/to...,The Toronto Raptors have agreed to a deal to a...
1,This is brand new territory for the Spurs,"Wed, 18 Jul 2018 09:46:13 EST",http://www.espn.com/nba/story/_/id/24128459/wi...,San Antonio is facing a significant transition...
2,Lakers' Hart named summer league MVP,"Tue, 17 Jul 2018 19:13:03 EST",http://www.espn.com/nba/story/_/id/24124114/lo...,"Lakers guard Josh Hart, averaging 24.2 points ..."
3,McDaniels leads Blazers (7-0) to Las Vegas title,"Wed, 18 Jul 2018 02:08:08 EST",http://www.espn.com/nba/story/_/id/24126169/po...,K.J. McDaniels scored a game-high 17 points to...
4,Lakers' Ball has arthroscopic surgery on knee,"Tue, 17 Jul 2018 21:14:50 EST",http://www.espn.com/nba/story/_/id/24124691/lo...,Lakers guard Lonzo Ball tried rest and PRP inj...
5,KD wins another award: Humanitarian of Year,"Wed, 18 Jul 2018 00:07:05 EST",http://www.espn.com/nba/story/_/id/24125658/go...,Warriors star Kevin Durant was named Sports Hu...
6,Cavaliers waive veteran center Perkins,"Tue, 17 Jul 2018 16:33:31 EST",http://www.espn.com/nba/story/_/id/24123368/cl...,The Cavaliers have waived center Kendrick Perk...
7,"Sources: Bjelica spurns 76ers, stays in Europe","Wed, 18 Jul 2018 09:36:42 EST",http://www.espn.com/nba/story/_/id/24121270/ne...,"Nemanja Bjelica, who had agreed to a one-year ..."
8,Can Jabari Parker get the happy ending Derrick...,"Sun, 15 Jul 2018 10:53:21 EST",http://www.espn.com/nba/story/_/id/24103566/ca...,Parker still holds the former MVP in high rega...
9,Rating the best and worst NBA moves so far,"Sun, 15 Jul 2018 13:33:23 EST",http://www.espn.com/nba/story/_/id/23997529/nb...,Kevin Pelton provides team-by-team analysis of...


In [5]:
ct = getRSS('http://www.chicagotribune.com/sports/rss2.0.xml')

In [6]:
ct['summary'][0]



'<p>Willson Contreras made history on Tuesday night with his third-inning home run off Blake Snell in the All-Star Game at Nationals Park.</p> <p>Contreras became the first player to homer on the first pitch of his major-league career as well as the first pitch he saw in an All-Star Game.</p> <p>“I’m so blessed,”...</p>'

In [7]:
def getsublink(link):
    z = urllib.request.urlopen(link).read().decode('utf-8')
    y = get_text(z)
    df = pd.DataFrame(columns=['link','data'])
    df = df.append({'link': link,'data': y}, ignore_index=True)
    resp = urllib.request.urlopen(link)
    soup = BeautifulSoup(resp, "lxml" , from_encoding=resp.info().get_param('charset'))
    try:
        for link in soup.find_all('a', href=True):
            if(link['href'][0] == 'h' and link['href'][1] == 't'):
                x = link['href']
                z = urllib.request.urlopen(x).read().decode('utf-8')
                y = get_text(z)
                df = df.append({'link': x,'data': y}, ignore_index=True)
    except:
        pass
    return df

output = getsublink("https://edition.cnn.com/2018/07/16/asia/india-whatsapp-lynching-intl/index.html")

In [8]:
output['data'][14]

"\nPlease enable JavaScript to view CNN Tech.\n\nbusinessculturegadgetsfuturestartupscnnmoney\nfuture\nBy Alanna Petroff\nChina is owning the future of cars. German automakers want inGerman automakers know the future of their industry is electric, autonomous and in China.\nBy Sherisse Pham\nGoogle's latest move to crack China: A WeChat game\nBy Charles Riley\nThis could be the Rolls-Royce of flying taxis\nBy Sara Ashley O'Brien\nLarry Page invests in another flying car startup\nBy Rishi Iyengar\nIndia now has the 'world's strongest' net neutrality rules"

In [9]:
output

,link,data
0,https://edition.cnn.com/2018/07/16/asia/india-...,Breaking News\nWorld+\nU.S.AfricaAmericasAsiaA...
1,http://money.cnn.com/INTERNATIONAL/,* Markets\n * Economy\n * Companies\n * T...
2,http://money.cnn.com/technology/,\nPlease enable JavaScript to view CNN Tech.\n...
3,http://go.cnn.com/?stream=cnni&sr=watchLiveHPb...,
4,http://money.cnn.com/INTERNATIONAL/,* Markets\n * Economy\n * Companies\n * T...
5,http://money.cnn.com/news/,* Companies\n * Markets\n * Tech\n * Medi...
6,http://money.cnn.com/data/markets/,* Companies\n * Markets\n * Tech\n * Medi...
7,http://money.cnn.com/technology/,\nPlease enable JavaScript to view CNN Tech.\n...
8,http://money.cnn.com/luxury/,* Companies\n * Markets\n * Tech\n * Medi...
9,http://money.cnn.com/media,\nPlease enable JavaScript to view CNN Media.\...


In [15]:
def preprocess_and_analyze(output,top_n=3):
    preprocsplit = (lambda rev: np.asarray(preprocess_string(rev)))
    text_rev = output['data'].apply(preprocsplit).values
    print(text_rev)
    
    common_dict = Dictionary(text_rev)
    word_list,raw_prob_list = [],[]
    for text in text_rev:
        count_dict = Counter(text)
        sum_words = sum(dict(count_dict).values()) #total number of 'parsed' words
        top_words = count_dict.most_common(top_n)
        if(top_words==[]): word_list.append([]), raw_prob_list.append([])
        else:
            words,commonalities = list(zip(*top_words))
            words = np.asarray(words)
            commonalities = np.around(np.log(np.asarray(commonalities).astype(float)/sum_words),decimals=4)
            word_list.append(words), raw_prob_list.append(commonalities)
    output['processed_text'] = pd.Series(text_rev)
    output['most_common_words'] = pd.Series(word_list)
    output['raw_probability'] = pd.Series(raw_prob_list)
    return output
    
output = preprocess_and_analyze(output)

[array(['break', 'new', 'world',
       'africaamericasasiaaustraliachinaeuropemiddl', 'eastuk', 'worldu',
       'politicsmoneyentertainmenttechsporttravelstylehealthvideovr',
       'live', 'search', 'intern', 'edit', 'intern', 'arab', 'español',
       'set', 'edit', 'prefer', 'internationalconfirm', 'intern', 'edit',
       'intern', 'arab', 'español', 'set', 'edit', 'prefer',
       'internationalconfirm', 'home', 'world', 'africa', 'america',
       'asia', 'australia', 'china', 'europ', 'middl', 'east', 'polit',
       'congress', 'suprem', 'court', 'kei', 'race', 'primari', 'result',
       'busi', 'market', 'tech', 'luxuri', 'star', 'screen', 'bing',
       'cultur', 'media', 'busi', 'cultur', 'gadget', 'futur', 'startup',
       'sport', 'world', 'cup', 'tenni', 'equestrian', 'golf', 'judo',
       'hors', 'race', 'motorsport', 'sail', 'footbal', 'rugbi', 'destin',
       'food', 'drink', 'plai', 'stai', 'video', 'art', 'design',
       'fashion', 'architectur', 'luxuri', 'au

In [16]:
print(output)
output.to_csv('outputCSV.csv')

                                                 link  \
0   https://edition.cnn.com/2018/07/16/asia/india-...   
1                 http://money.cnn.com/INTERNATIONAL/   
2                    http://money.cnn.com/technology/   
3   http://go.cnn.com/?stream=cnni&sr=watchLiveHPb...   
4                 http://money.cnn.com/INTERNATIONAL/   
5                          http://money.cnn.com/news/   
6                  http://money.cnn.com/data/markets/   
7                    http://money.cnn.com/technology/   
8                        http://money.cnn.com/luxury/   
9                          http://money.cnn.com/media   
10                   http://money.cnn.com/technology/   
11          http://money.cnn.com/technology/business/   
12           http://money.cnn.com/technology/culture/   
13           http://money.cnn.com/technology/gadgets/   
14            http://money.cnn.com/technology/future/   
15          http://money.cnn.com/technology/startups/   
16  https://www.turnerjobs.com/